In [1]:
import pprint
import tempfile

from IPython import display
import matplotlib.pyplot as plt

import tensorflow as tf
import orbit
import tensorflow_models as tfm
import tensorflow_datasets as tfds

from official.core import exp_factory
from official.core import config_definitions as cfg
from official.vision.serving import export_saved_model_lib
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder

pp = pprint.PrettyPrinter(indent=4) # Set Pretty Print Indentation
print(tf.__version__) # Check the version of tensorflow used

%matplotlib inline

2025-02-27 05:38:44.216615: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-27 05:38:44.231352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740634724.250029    7092 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740634724.255485    7092 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 05:38:44.273772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

2.18.0


In [17]:
BATCH_SIZE = 128
SHUFFLE = 1024
NUM_CLASSES = 10


IMG_SIZE = [128,128,3]

train_data_input_path = '/home/atallah/tensorflow_datasets/cifar10/3.0.2/cifar10-train.tfrecord-00000-of-00001'
valid_data_input_path = '/home/atallah/tensorflow_datasets/cifar10/3.0.2/cifar10-test.tfrecord-00000-of-00001'
model_dir = './trained_model/'

In [21]:
exp_config = exp_factory.get_exp_config('retinanet_resnetfpn_coco')

# Backbone config.
exp_config.task.freeze_backbone = False
exp_config.task.annotation_file = ''

# Model config.
exp_config.task.model.input_size = IMG_SIZE
exp_config.task.model.num_classes = NUM_CLASSES + 1
exp_config.task.model.detection_generator.tflite_post_processing.max_classes_per_detection = exp_config.task.model.num_classes

# Training data config.
exp_config.task.train_data.input_path = train_data_input_path
exp_config.task.train_data.dtype = 'float32'
exp_config.task.train_data.global_batch_size = BATCH_SIZE
exp_config.task.train_data.parser.aug_scale_max = 1.0
exp_config.task.train_data.parser.aug_scale_min = 1.0

# Validation data config.
exp_config.task.validation_data.input_path = valid_data_input_path
exp_config.task.validation_data.dtype = 'float32'
exp_config.task.validation_data.global_batch_size = BATCH_SIZE 

In [22]:
train_steps = 1000
exp_config.trainer.steps_per_loop = 100 # steps_per_loop = num_of_training_examples // train_batch_size

exp_config.trainer.summary_interval = 100
exp_config.trainer.checkpoint_interval = 100
exp_config.trainer.validation_interval = 100
exp_config.trainer.validation_steps =  100 # validation_steps = num_of_validation_examples // eval_batch_size
exp_config.trainer.train_steps = train_steps
exp_config.trainer.optimizer_config.warmup.linear.warmup_steps = 100
exp_config.trainer.optimizer_config.learning_rate.type = 'cosine'
exp_config.trainer.optimizer_config.learning_rate.cosine.decay_steps = train_steps
exp_config.trainer.optimizer_config.learning_rate.cosine.initial_learning_rate = 0.1
exp_config.trainer.optimizer_config.warmup.linear.warmup_learning_rate = 0.05

In [23]:
pp.pprint(exp_config.as_dict())
display.Javascript('google.colab.output.setIframeHeight("500px");')

{   'runtime': {   'all_reduce_alg': None,
                   'batchnorm_spatial_persistent': False,
                   'dataset_num_private_threads': None,
                   'default_shard_dim': -1,
                   'distribution_strategy': 'mirrored',
                   'enable_xla': False,
                   'gpu_thread_mode': None,
                   'loss_scale': None,
                   'mixed_precision_dtype': 'bfloat16',
                   'num_cores_per_replica': 1,
                   'num_gpus': 0,
                   'num_packs': 1,
                   'per_gpu_thread_count': 0,
                   'run_eagerly': False,
                   'task_index': -1,
                   'tpu': None,
                   'tpu_enable_xla_dynamic_padder': None,
                   'use_tpu_mp_strategy': False,
                   'worker_hosts': None},
    'task': {   'allow_image_summary': False,
                'annotation_file': '',
                'differential_privacy_config': None,
     

<IPython.core.display.Javascript object>

In [9]:
logical_device_names = [logical_device.name for logical_device in tf.config.list_logical_devices()]
print(logical_device_names)

if exp_config.runtime.mixed_precision_dtype == tf.float16:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

if 'GPU' in ''.join(logical_device_names):
  distribution_strategy = tf.distribute.MirroredStrategy()

else:
  print('Warning: this will be really slow.')
  distribution_strategy = tf.distribute.OneDeviceStrategy(logical_device_names[0])

print('Done')

with distribution_strategy.scope():
  task = tfm.core.task_factory.get_task(exp_config.task, logging_dir=model_dir)

['/device:CPU:0', '/device:GPU:0']
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Done


In [24]:
model, eval_logs = tfm.core.train_lib.run_experiment(
    distribution_strategy=distribution_strategy,
    task=task,
    mode='train_and_eval',
    params=exp_config,
    model_dir=model_dir,
    run_post_eval=True)

restoring or initializing model...
INFO:tensorflow:Customized initialization is done through the passed `init_fn`.


INFO:tensorflow:Customized initialization is done through the passed `init_fn`.


train | step:      0 | training until step 100...
INFO:tensorflow:Error reported to Coordinator: Exception encountered when calling layer 'retina_net_model_2' (type RetinaNetModel).

in user code:

    File "/home/atallah/miniconda3/envs/tensorflow18/lib/python3.12/site-packages/official/vision/modeling/retinanet_model.py", line 129, in call  *
        features = self.backbone(images)
    File "/home/atallah/miniconda3/envs/tensorflow18/lib/python3.12/site-packages/tf_keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/atallah/miniconda3/envs/tensorflow18/lib/python3.12/site-packages/tf_keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "res_net_2" is incompatible with the layer: expected shape=(None, 32, 32, 3), found shape=(128, 128, 128, 3)


Call arguments received by layer 'retina_net_model_2' (type RetinaNetModel):


INFO:tensorflow:Error reported to Coordinator: Exception encountered when calling layer 'retina_net_model_2' (type RetinaNetModel).

in user code:

    File "/home/atallah/miniconda3/envs/tensorflow18/lib/python3.12/site-packages/official/vision/modeling/retinanet_model.py", line 129, in call  *
        features = self.backbone(images)
    File "/home/atallah/miniconda3/envs/tensorflow18/lib/python3.12/site-packages/tf_keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/atallah/miniconda3/envs/tensorflow18/lib/python3.12/site-packages/tf_keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "res_net_2" is incompatible with the layer: expected shape=(None, 32, 32, 3), found shape=(128, 128, 128, 3)


Call arguments received by layer 'retina_net_model_2' (type RetinaNetModel):
  • images=tf.Tensor(shape=(128, 128, 128, 3), dty

ValueError: in user code:

    File "/home/atallah/miniconda3/envs/tensorflow18/lib/python3.12/site-packages/orbit/utils/loop_fns.py", line 120, in loop_fn  *
        step_fn(iterator)
    File "/home/atallah/miniconda3/envs/tensorflow18/lib/python3.12/site-packages/official/core/base_trainer.py", line 391, in step_fn  *
        logs = task_train_step(
    File "/home/atallah/miniconda3/envs/tensorflow18/lib/python3.12/site-packages/official/vision/tasks/retinanet.py", line 327, in train_step  *
        outputs = model(features, training=True)
    File "/home/atallah/miniconda3/envs/tensorflow18/lib/python3.12/site-packages/tf_keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filepciat_8d.py", line 47, in tf__call
        features = ag__.converted_call(ag__.ld(self).backbone, (ag__.ld(images),), None, fscope)

    ValueError: Exception encountered when calling layer 'retina_net_m